# Logistic Regression (Ridge)

In [1]:
import sys
sys.path.insert(1, '../../libs')
from utils import get_data, temporal_train_test_split
from autorship import AuthorClassifier
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import TweetTokenizer
import numpy as np
import pandas as pd
import re

In [2]:
data = get_data("../../data/authors.csv")

In [3]:
tt = TweetTokenizer()

vectorizers = [CountVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,5), analyzer="char"),
                CountVectorizer(ngram_range=(4,5), analyzer="char"),
                CountVectorizer(ngram_range=(3,8), analyzer="char"),
                TfidfVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(4,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(3,8), analyzer="char")]

In [4]:
from joblib import Parallel, delayed
def process(vectorizer):
    evaluation = list()
    usernames = list(np.unique(data["username"]))
    
    vectorizer_str = vectorizer.__str__()
    print(f"Running => {vectorizer_str}")

    for i in range(len(usernames)):
        author1 = usernames.pop()

        for author2 in usernames:
            X_train, X_test, y_train, y_test = temporal_train_test_split(
                data, author1, author2)

            clf = AuthorClassifier(vectorizer, clf=LogisticRegression(random_state=42, penalty="l2", solver="liblinear"), scaler=MaxAbsScaler())
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            evaluation.append(clf.evaluate(y_test, y_pred))
            metrics = pd.DataFrame(evaluation)[["f1_macro", "recall_macro", "precision_macro", "accuracy", "auc_score"]].mean()
    
    return (vectorizer_str, metrics)
    
results = Parallel(n_jobs=6)(delayed(process)(vectorizer) for vectorizer in vectorizers)
print(results) 

Running => CountVectorizer(analyzer='char', ngram_range=(1, 5))
Running => CountVectorizer(analyzer='char', ngram_range=(4, 5))
Running => CountVectorizer(analyzer='char', ngram_range=(3, 8))
Running => TfidfVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f4e9655dcd0>>)
Running => CountVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f5110515cd0>>)
Running => CountVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fefe58fbcd0>>)
Running => TfidfVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f50fc733a00>>)
Running => TfidfVectorizer(analyzer='char', ngram_range=(1, 5))
Running => TfidfVectorizer(analyzer='char', ngram_range=(4, 5))
Running => TfidfVectorizer(analyzer=

In [5]:
metrics_df = pd.DataFrame([results[i][1] for i in range(len(results))])
metrics_df["vectorizer"] = [results[i][0] for i in range(len(results))]
metrics_df.to_csv("../../results/logistic_regression_l2.csv")
metrics_df

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.833431,0.835007,0.841118,0.834176,0.920060,CountVectorizer(tokenizer=<bound method TweetT...
1,0.806137,0.811221,0.833063,0.809655,0.922682,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.839260,0.840470,0.844441,0.839736,0.920372,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.822369,0.824104,0.829835,0.823210,0.907170,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.819774,0.822150,0.830653,0.821056,0.910069,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.872112,0.872323,0.873116,0.872254,0.941017,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.852171,0.852767,0.858619,0.853072,0.935113,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.857850,0.857998,0.858894,0.858037,0.933443,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.837922,0.838253,0.839309,0.838135,0.916890,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.840254,0.840533,0.842274,0.840578,0.919463,"TfidfVectorizer(analyzer='char', ngram_range=(..."


Row with max sum of columns (remember that by the result of f1-score we use the precision and recall score)

In [6]:
print(metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()])
print("\nVectorizer:", metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()]["vectorizer"])

f1_macro                                                    0.872112
recall_macro                                                0.872323
precision_macro                                             0.873116
accuracy                                                    0.872254
auc_score                                                   0.941017
vectorizer         TfidfVectorizer(tokenizer=<bound method TweetT...
Name: 5, dtype: object

Vectorizer: TfidfVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f4e9655dcd0>>)


In [7]:
pd.read_csv("../../results/logistic_regression_l2.csv").drop("Unnamed: 0", axis=1)

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.833431,0.835007,0.841118,0.834176,0.920060,CountVectorizer(tokenizer=<bound method TweetT...
1,0.806137,0.811221,0.833063,0.809655,0.922682,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.839260,0.840470,0.844441,0.839736,0.920372,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.822369,0.824104,0.829835,0.823210,0.907170,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.819774,0.822150,0.830653,0.821056,0.910069,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.872112,0.872323,0.873116,0.872254,0.941017,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.852171,0.852767,0.858619,0.853072,0.935113,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.857850,0.857998,0.858894,0.858037,0.933443,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.837922,0.838253,0.839309,0.838135,0.916890,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.840254,0.840533,0.842274,0.840578,0.919463,"TfidfVectorizer(analyzer='char', ngram_range=(..."
